In [1]:
!pip install /kaggle/input/objectdetectionlibrary/ObjectDetectionLibrary/object_detection-2.4/dist/object_detection-0.1.tar

In [2]:
!mkdir /kaggle/working/out

In [3]:
!python /kaggle/input/objectdetectionlibrary/ObjectDetectionLibrary/generate_label_map.py --indir /kaggle/input/playing-cards-dataset/train_zipped --outdir /kaggle/working/out/label_map.pbtxt 

In [4]:
!mkdir /kaggle/working/plot

In [5]:
#!python /kaggle/input/objectdetectionlibrary/ObjectDetectionLibrary/plotBoxes.py --indir /kaggle/input/playing-cards-dataset/test_zipped --anot /kaggle/input/playing-cards-dataset/test_zipped --outdir /kaggle/working/plot --labels /kaggle/working/out/label_map.pbtxt  

In [6]:
!python /kaggle/input/objectdetectionlibrary/ObjectDetectionLibrary/generate_tfrecord.py -x /kaggle/input/playing-cards-dataset/test_zipped -l /kaggle/working/out/label_map.pbtxt -o /kaggle/working/out/test.record -i /kaggle/input/playing-cards-dataset/test_zipped  

In [7]:
!python /kaggle/input/objectdetectionlibrary/ObjectDetectionLibrary/generate_tfrecord.py -x /kaggle/input/playing-cards-dataset/train_zipped -l /kaggle/working/out/label_map.pbtxt -o /kaggle/working/out/train.record -i /kaggle/input/playing-cards-dataset/train_zipped  

In [8]:
!mkdir /kaggle/working/out/model_ckpt

In [9]:
!python /kaggle/input/objectdetectionlibrary/ObjectDetectionLibrary/model_main_tf2.py --model_dir /kaggle/working/out/model_ckpt --pipeline_config_path /kaggle/input/fasterrcnn/pipeline.config --num_train_steps 3000

In [10]:
 !python /kaggle/input/objectdetectionlibrary/ObjectDetectionLibrary/model_main_tf2.py --model_dir /kaggle/working/out/model_ckpt --pipeline_config_path /kaggle/input/fasterrcnn/pipeline.config --checkpoint_dir /kaggle/working/out/model_ckpt

In [11]:
!python /kaggle/input/objectdetectionlibrary/ObjectDetectionLibrary/exporter_main_v2.py --input_type image_tensor --pipeline_config_path /kaggle/input/fasterrcnn/pipeline.config --trained_checkpoint_dir /kaggle/working/out/model_ckpt --output_directory /kaggle/working/out/out_model

In [13]:
import tensorflow as tf
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as viz_utils
import cv2
import numpy as np

In [14]:
detect_fn=tf.saved_model.load("/kaggle/working/out/out_model/saved_model")

In [16]:
category_index=label_map_util.create_category_index_from_labelmap("/kaggle/working/out/label_map.pbtxt", use_display_name=True)

In [17]:
category_index

In [18]:
img = cv2.imread("/kaggle/input/playing-cards-dataset/test_zipped/003212838.jpg")
img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
img= np.array([img])

In [19]:
img.shape

In [20]:
detections=detect_fn(img)

In [21]:
detections

In [24]:
boxes = detections['detection_boxes'].numpy()[0]
classes=np.uint8(detections['detection_classes'].numpy()[0])
scores = detections['detection_scores'].numpy()[0]

In [25]:
boxes=boxes[scores>0.5]
classes=classes[scores>0.5]
scores=scores[scores>0.5]

In [26]:
boxes.shape

In [34]:
scores

In [31]:
image_detections=img[0,:,:,:].copy()

In [32]:
viz_utils.visualize_boxes_and_labels_on_image_array(
        image_detections,
        boxes,
        classes,
        scores,
        category_index,
        use_normalized_coordinates=True
)

In [33]:
cv2.imwrite("/kaggle/working/prediction.jpg",image_detections)